In [1]:
# ! pip install -q -r requirements.txt -U
# # ! pip install -q torch numpy tqdm

In [2]:
! pip install open-clip-torch==0.2.1

In [3]:
import numpy, scipy, sklearn, pandas
numpy.__version__, scipy.__version__, sklearn.__version__, pandas.__version__


('1.26.4', '1.16.3', '1.8.0', '2.3.3')

In [4]:
from PrismSSL.multimodal.Trainer import Trainer

/opt/anaconda3/envs/test-prism/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/test-prism/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/opt/anaconda3/envs/test-prism/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribu

In [5]:
import torch
from torch.utils.data import Dataset

class DummyMultimodalDataset(Dataset):
    def __init__(
        self,
        num_samples=10,
        audio_length=16000,
        image_channels=3,
        image_height=224,
        image_width=224,
    ):
        self.num_samples = num_samples

        # Pre-generate fixed-size dummy data
        self.audio = torch.randn(num_samples, audio_length)
        self.images = torch.randn(
            num_samples, image_channels, image_height, image_width
        )

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return {
            "audio": self.audio[idx],   # [audio_length]
            "image": self.images[idx],  # [C, H, W]
        }


In [6]:
dataset = DummyMultimodalDataset()

sample = dataset[0]
print(sample.keys())
print(sample["audio"].shape)   # torch.Size([16000])
print(sample["image"].shape)   # torch.Size([3, 224, 224])


dict_keys(['audio', 'image'])
torch.Size([16000])
torch.Size([3, 224, 224])


In [7]:
trainer = Trainer(
    method = 'wav2clip',
    backbone = None,
    # feature_size=128,
    wandb_project = 'eat-pretext-1',
    wandb_entity = None,
    wandb_mode = "online", # "online", "offline", "disabled"
    wandb_run_name = "With labled LibriSpeech dataset-1",
    wandb_config = None,
    wandb_notes = None,
    wandb_tags = None,
    variant = 'base',
    checkpoint_interval= 5,
    num_workers=0,
    )

2025-12-23 20:29:38 | INFO | 
---------------- PrismSSL: Multimodal ----------------
Number of workers : 0
Number of GPUs    : 0
Device            : cpu
Method            : wav2clip

2025-12-23 20:29:39 | INFO | 
---------------- Wav2CLIP Configuration ----------------
Audio Encoder                    : Wav2ClipAudioEncoder
Image Encoder                    : CLIPImageEncoder
Projection Dimension             : 512
Image Encoder Frozen             : No
Contrastive Learning             : Audio ↔ Image modality alignment
Loss                             : Contrastive (user-defined externally)

2025-12-23 20:29:39 | INFO | 
---------------- Model Parameters ----------------
Total Parameters : 163,235,521
--------------------------------------------------
/Users/kianoosh/Desktop/PrismSSL/PrismSSL/PrismSSL/multimodal/Trainer.py:196: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.Grad

In [8]:
trainer.train(
    train_dataset = dataset,
    optimizer='adamw',
    epochs=1,
    use_embedding_logger=False,
    # logger_loader=logger_loader,
    batch_size=8,
    use_hpo=False,
    n_trials=1,
    tuning_epochs=5,
    lr=3e-5,
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


ERROR:WandbLogger:An unexpected error occurred during W&B initialization: Error uploading run: returned error 403: {"data":null,"errors":[{"message":"\n\u003chtml\u003e\u003chead\u003e\n\u003cmeta http-equiv=\"content-type\" content=\"text/html;charset=utf-8\"\u003e\n\u003ctitle\u003e403 Forbidden\u003c/title\u003e\n\u003c/head\u003e\n\u003cbody text=#000000 bgcolor=#ffffff\u003e\n\u003ch1\u003eError: Forbidden\u003c/h1\u003e\n\u003ch2\u003eYour client does not have permission to get URL \u003ccode\u003e/graphql\u003c/code\u003e from this server.\u003c/h2\u003e\n\u003ch2\u003e\u003c/h2\u003e\n\u003c/body\u003e\u003c/html\u003e\n"}]}
2025-12-23 20:29:44 | INFO | W&B logging is not active for this run.
2025-12-23 20:29:44 | WARNING | [Dataset Check] Your dataset should return all 'audio', 'image' and 'text'  keys. Currently missing: text
Wav2CLIP Training:   0%|          | 0/1 [00:00<?, ?epoch/s]/Users/kianoosh/Desktop/PrismSSL/PrismSSL/PrismSSL/multimodal/Trainer.py:824: FutureWarning: 